# N_Queen

Descrizione del problema:
- Stati: una qualsiasi disposizione di n regine, in modo tale che ci sia una sola regina per colonna e una sola regina per riga.
- Goal State: una qualsiasi disposizione delle n regine sulla scacchiera che non si attaccano a vicenda.
- Funzione successore: un qualsiasi scambio di due colonne della scacchiera.
- Funzione di valutazione: numero di attacchi.
- Test obiettivo: numero di attacchi uguale a zero. 

Algoritmi applicati:
- Steepest Ascent Hill Climbing
- Random-Restart Hill Climbing (Iterated Hill Climbing) 
- Stochastic Hill Climbing
- Simulated Annealing

# Import

In [1]:
import random
import math

# Definizione dello Stato

Variabili:
- lst
- dim
- val

Funzioni:
- tweek
- print

In [2]:
class State:
    
    def __init__(self, lst, val):
        self.dim = len(lst)
        self.lst = lst
        self.val = val
    
    def randomTweek(self):
        i = random.randint(0, self.dim -1)
        j = random.randint(0, self.dim -1)
        while(i == j):
            j = random.randint(0, self.dim -1)
            
        new_lst = self.lst.copy()
        new_lst[i],new_lst[j] = new_lst[j],new_lst[i]
        return new_lst
    
    def tweek(self, i, j):
        new_lst = self.lst.copy()
        new_lst[i],new_lst[j] = new_lst[j],new_lst[i]
        return new_lst
    
    def __str__(self) -> str:
        out = ""
        for i in range(self.dim):
            row = ""
            for j in range(self.dim):
                if self.lst[i] == j:
                    row += " Q "
                else:
                    row += " . "
            out += (row + "\n")
        return out


# Game Definition

Var:
- initial_state
- n_queens

Fun:
- randomStart
- makeTweek
- evalState
- goalCheck

In [3]:
class Game:
    
    def __init__(self, n_queens):
        self.n_queens = n_queens
        self.initial_state = self.randomStart()
        
    
    def randomStart(self):
        layout = list(range(self.n_queens))
        
        for _ in range(self.n_queens):
            i = random.randint(0, self.n_queens -1)
            j = random.randint(0, self.n_queens -1)
            while(i == j):
                j = random.randint(0, self.n_queens -1)
            layout[i],layout[j] = layout[j],layout[i]
        
        return State(layout, self.evalState(layout))

    def makeRandomTweek(self, state):
        layout = state.randomTweek()
        return State(layout, self.evalState(layout))
    
    def makeTweek(self, state, i, j):
        layout = state.tweek(i,j)
        return State(layout, self.evalState(layout))
    
    def evalState(self, layout):
        collision = 0
        dx = [-1,1,-1,1]
        dy = [-1,1,1,-1]

        for i in range(self.n_queens):  
            for j in range(0,4):
                row = i + dx[j]
                col = layout[i] + dy[j]
                
                while(0<= row < self.n_queens and 0 <= col < self.n_queens):
                    if(layout[row] == col):
                        collision+=1
                        
                    row = row + dx[j]
                    col = col + dy[j]
        
        return collision
    
    def goalCheck(self, state):
        return self.evalState(state) == 0

In [4]:
game = Game(8)
#print(game.initial_state.lst, game.initial_state.val)


# Steepest Ascent Hill Climbing

Specifiche:

- function STEEPEST-ASCENT(problem) returns uno stato che è un massimo locale

- inputs: problem, un problema

- local variables: current, un nodo next, un nodo

Pseudocodice:

    current ⟵ MAKE-NODE(INITIAL-STATE(problem))

    loop do

        next ⟵ successore di current di valore più alto

        if VALUE(next) >= VALUE(current) then return STATE(current)

        current ⟵ next

    end

In [5]:
def steepestAscent(game, initial_state):
    curr = initial_state
    
    while(True):
        
        local_best = None
        for i in range(game.n_queens -1):
            for j in range (i+1, game.n_queens -1):
                next = game.makeTweek(curr, i, j)
                if (local_best == None or next.val < local_best.val):
                    local_best = next
        
        if(local_best.val >= curr.val):
            return local_best
        
        curr = local_best

In [6]:
sol = steepestAscent(game, game.initial_state)
print(sol.lst, sol.val)

[2, 4, 1, 5, 3, 6, 0, 7] 4


# Random-Restart Hill Climbing (Iterated Hill Climbing) 

Specifiche:

- function RANDOM-RESTART-HILL-CLIMBING(problem) returns uno stato soluzione

- inputs: problem, un problema

- local variables: current, next, best: nodi; local: booleana

Pseudocodice:

    t ⟵ 0

    Inizializza best

    repeat

        local ⟵ true

        seleziona un punto iniziale current random

        repeat

            next ⟵ successore di current con VALUE più alto

            if VALUE(next) < VALUE(current)

                then current ⟵ next

                else local ⟵ false

        until local

        t ⟵ t + 1

        if VALUE(current) < VALUE(best) then best ⟵ current

    until t = MAX

    return STATE(best)
    
    end

In [7]:
def randomRestart(game, max_iter):
    curr_iter = 0
    global_best = game.initial_state
    
    while(curr_iter < max_iter and global_best.val > 0):
        local_best = steepestAscent(game, game.randomStart())
        if(local_best.val < global_best.val):
            global_best = local_best
        curr_iter += 1
    
    return global_best


In [8]:
game = Game(20)
sol = randomRestart(game, 800)
print(sol.lst, sol.val)



[15, 17, 7, 4, 8, 11, 19, 6, 2, 12, 1, 18, 10, 13, 9, 5, 3, 0, 14, 16] 0


# Stochastic Hill-Climbing

Specifiche:
- function STOCHASTIC-HILL-CLIMBING(problem) returns uno stato soluzione
- inputs: problem, un problema
- local variables: current, next, best: nodi

Osservazioni:
- ∆E = VALUE(next) - VALUE(current)
- al crescere di T diventa sempre meno importante la differenza delle valutazioni dei due stati ai fini dell’accettazione dello stato next
- Se invece T è molto piccolo (e.g., T = 1) la procedura diventa simile ad un semplice hill-climber.

Pseudocodice:

    t ⟵ 0
    
    seleziona un punto iniziale current random
    
    Inizializza best
    
    repeat
    
        next ⟵ seleziona successore di current random
    
        seleziona next come nodo corrente con probabilità p = 1/(1 + e∆E/T)
    
        se selezionato next come nuovo current:
    
            if VALUE(current) < VALUE(best) then best ⟵ current
    
            t ⟵ t + 1
    
    until t = MAX
    
    return STATE(best)
    
    end

In [9]:
def stochasticHill(game, max_iter, t):
    curr_iter = 0
    global_best = game.initial_state
    curr = game.initial_state
    
    while(curr_iter < max_iter and global_best.val > 0):
        next = game.makeRandomTweek(curr)
        
        dE = next.val - curr.val 
        chosing_prob = 1 / (1 + math.exp(dE / t))
        
        if(random.random() < chosing_prob):
            curr = next
            if(curr.val < global_best.val):
                global_best = curr
        
        curr_iter += 1
        
    return global_best

In [10]:
game = Game(8)
sol = stochasticHill(game, 2000000, 8)

print(sol.lst, sol.val)
print(sol)

[3, 6, 0, 7, 4, 1, 5, 2] 0
 .  .  .  Q  .  .  .  . 
 .  .  .  .  .  .  Q  . 
 Q  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  Q 
 .  .  .  .  Q  .  .  . 
 .  Q  .  .  .  .  .  . 
 .  .  .  .  .  Q  .  . 
 .  .  Q  .  .  .  .  . 



# Simulated Annealing

Specifiche:
- function SIMULATED-ANNEALING(problem) returns uno stato soluzione
- inputs: problem, un problema
- local variables: current, next: nodi;

Osservazioni
- T “temperatura” iniziale 
- g(T,t) rapporto di raffreddamento 
- condizione di terminazione (max step for change)
- goal_check (halting-criterion)
- ∆E = VALUE(next) - VALUE(current)

Pseudocodice:
    
    t ⟵ 0
    
    Inizializza T
    
    seleziona un punto iniziale current random
    
    nizializza best
    
    repeat
    
        repeat
    
            next ⟵ successore di current scelto random
    
            if VALUE(next) > VALUE(current)
    
                then current ⟵ next
    
            if VALUE(current) > VALUE(best) then best ⟵ current
    
                else if random[0, 1) < e-∆E/T then current ⟵ next
    
        until (termination-condition)
    
        T = g(T, t)
    
        t ⟵ t + 1
    
    until (halting-criterion)
    
    return STATE(best)
    
    end

In [11]:
def simulatedAnnealing(game, starting_temp, ending_temp, cooling_ratio, iter_for_change):
    temp = starting_temp
    curr = game.initial_state
    global_best = game.initial_state
    
    while(temp > ending_temp and global_best.val > 0):
        curr_iter = 0
        
        while(curr_iter < iter_for_change):
            next = game.makeRandomTweek(curr)
            flag = False
            
            if(next.val < curr.val):
                curr = next
                flag = True
            else:
                dE = next.val - curr.val    
                chosing_prob = math.exp(-dE/temp)
                if(random.random() < chosing_prob):
                    curr = next
                    flag = True
            
            if(curr.val < global_best.val and flag):
                global_best = curr
            
            curr_iter += 1
        
        temp = temp * cooling_ratio
    
    return global_best

In [12]:
game = Game(20)
sol = simulatedAnnealing(game, 30, 0.5, 0.9, 100)
print(sol.lst, sol.val)

[14, 9, 13, 1, 8, 17, 0, 16, 5, 2, 10, 6, 4, 12, 15, 11, 18, 7, 3, 19] 2


# Tabu Search

Pseudocodice:

    function tabu_search(problem)
    
    Definizione tabu tenure
    
    Inizializza la tabu list M
    
    current ⟵ un certo stato iniziale in S
    
    Inizializza best
    
    repeat
    
        Calcolo del neighborhood N di current
    
        current ⟵ migliore stato ∈ non_tabu_N
    
        update(M)
    
        best ⟵ migliore(current, best)
    
    until condizione-terminazione
    
    return STATE(best)

In [424]:
def tabuSearch(game, tabu_tenure, max_iter):
    tabu_list = {}
    curr_iter = 0
    curr = game.initial_state
    global_best = game.initial_state
    
    while(curr_iter < max_iter and global_best.val > 0):

        local_best = None
        for i in range(game.n_queens):
            for j in range (i+1, game.n_queens):
                next = game.makeTweek(curr, i, j)
                if((i,j) not in tabu_list and (local_best == None or next.val < local_best.val)):
                    local_best = next
                    move = (i,j)
        
        curr = local_best
        
        if(local_best.val < global_best.val):
            global_best = local_best
        
        for key in tabu_list:
            tabu_list[key] -= 1
            
        tabu_list = {k: tabu_list[k] for k in tabu_list if tabu_list[k]>0}   
        tabu_list[move] = tabu_tenure
        
        curr_iter += 1
    
    return global_best                    

In [425]:
game = Game(8)
game.initial_state = State([7, 5, 4, 0, 1, 3, 6, 2], 4)
sol = tabuSearch(game, 5, 100)

print(sol.lst, sol.val)

[5, 0, 4, 1, 7, 2, 6, 3] 0


# Tabu Search con l’Aspiration Criterion

Pseudocodice:

    function tabu_search(problem)
    
    Definizione tabu tenure
    
    Inizializza la tabu list M
    
    current ⟵ un certo stato iniziale in S
    
    Inizializza best
    
    repeat
    
        Calcolo del neighborhood N di current
    
        Calcolo di A
    
        current ⟵ migliore stato ∈ {non_tabu_N ∪ A}
    
        update(M)
    
        best ⟵ migliore(current, best)
    
    until condizione-terminazione
    
    return STATE(best)

In [430]:
def tabuSearchAC(game, tabu_tenure, max_iter):
    tabu_list = {}
    curr_iter = 0
    curr = game.initial_state
    global_best = game.initial_state
    
    while(curr_iter < max_iter and global_best.val > 0):
        
        local_best = None
        for i in range(game.n_queens):
            for j in range (i+1, game.n_queens):
                next = game.makeTweek(curr, i, j)
                
                if((i,j) not in tabu_list and (local_best == None or next.val < global_best.val)):
                    local_best = next
                    move = (i,j)
                
                if(local_best != None and next.val < local_best.val and (i,j) not in tabu_list):
                    local_best = next
                    move = (i,j)
        
        curr = local_best
        
        if(local_best.val < global_best.val):
            global_best = local_best
            
        for key in tabu_list:
            tabu_list[key] -= 1
            
        tabu_list = {k: tabu_list[k] for k in tabu_list if tabu_list[k]>0}
        tabu_list[move] = tabu_tenure
        
        curr_iter += 1
    
    return global_best

In [441]:
game = Game(8)
sol = tabuSearchAC(game, 5, 500)

print(sol.lst, sol.val)

[5, 7, 1, 3, 0, 6, 4, 2] 0


# Tabu Search Frequency-Based Memory

In [442]:
def tabuSearchFBM(game, tabu_tenure, max_iter, moves_knowledge, penality_ratio):
    global_best = game.initial_state
    curr = game.initial_state
    curr_iter = 0
    
    tabu_list = {}
    fb_meory = {}
    last_moves = []
    
    while(curr_iter < max_iter and global_best.val > 0):
        
        local_best = None
        for i in range(game.n_queens):
            for j in range(i+1, game.n_queens):
                next = game.makeTweek(curr, i, j)
                
                if((i,j) not in tabu_list and (local_best == None or next.val < global_best.val)):
                    local_best = next
                    move = (i,j)
                
                if(local_best != None and (next.val - penality_ratio * fb_meory.get((i,j),0)) < (local_best.val - penality_ratio * fb_meory.get((i,j),0)) and (i,j) not in tabu_list):
                    local_best = next
                    move = (i,j)
                
        curr = local_best
        
        if(local_best.val < global_best.val):
            global_best = local_best
            
        if(len(last_moves) == moves_knowledge):
            fb_meory[last_moves.pop(0)] -= 1
            
        fb_meory[move] = fb_meory.get(move, 0) + 1
        last_moves.append(move)
            
            
        for key in tabu_list:
            tabu_list[key] -= 1
        
        tabu_list = {k: tabu_list[k] for k in tabu_list if tabu_list[k] > 0}
        tabu_list[move] = tabu_tenure
        
        curr_iter += 1
    
    return global_best
    

In [454]:
game = Game(20)
sol = tabuSearchFBM(game, 5, 200, 70, 0.7)

print(sol.lst, sol.val)

[3, 10, 0, 19, 1, 12, 14, 11, 8, 5, 2, 16, 18, 6, 15, 17, 7, 4, 13, 9] 0
